In [2]:
!pip install seaborn

In [6]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from math import floor
from dtw import dtw
#from matrixprofile.discover import snippets
import pandas as pd
from pyts.image import RecurrencePlot
#import tensorflow as tf
import json
from sklearn.model_selection import train_test_split
import random

In [7]:
def from_np_array(array_string):
    array_string = ','.join(array_string.replace('[ ', '[').split())
    return np.array(ast.literal_eval(array_string))

In [16]:
p = Path("D:\\Study\\SANNI\\Dataset\\WalkRun1_80_3800_200.csv")

df = pd.read_csv(p,converters={"neighbors":json.loads})

In [48]:
df

,Unnamed: 0,key,snippet,fraction,neighbors
0,0,0,[0.55714116 0.56319213 0.56522101 0.57919439 0...,0.311884,"[[0.014993, -0.043289, -0.0039098, -0.038406, ..."
1,1,1,[0.47102721 0.44296578 0.4652721 0.50352454 0...,0.430870,"[[7.0488, 4.9723, 4.6134, 3.3802, 2.4563, 2.16..."
2,2,2,[0.54415993 0.545161 0.54891279 0.55129218 0...,0.257536,"[[-0.038406, 0.001217, 0.0039053, -0.0056399, ..."


In [18]:
    subseq_count = [(i, len(np.array(df.neighbors.iloc[i]))) for i in range(0, len(df.neighbors))]
    max_subseq_count = max([subseq_count[i][1] for i in range(0, len(subseq_count))])

<ipython-input-18-44bfef0de257>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  subseq_count = [(i, len(np.array(df.neighbors.iloc[i]))) for i in range(0, len(df.neighbors))]


In [19]:
#в долях
e = 0.01

In [90]:
def augmentation(data: pd.DataFrame, e=0.01):
    """
    Увеличение и балансировка соседей.
    Все соседи сниппетов увеличиваются до количеста соседей у сниппета с максиальным fraction
    :param data: dataframe, в котором хранятся сниппеты и их соседи
    :param e: 0<e<1 процент, на который можно сдвинуть точку
    :return: возвращается dataframe той же структуры, но со сбалансированными соседями
    """
    subseq_count = [(i, len(np.array(data.neighbors.iloc[i]))) for i in range(0, len(data.neighbors))]
    max_subseq_count = max([subseq_count[i][1] for i in range(0, len(subseq_count))])

    new_neighbors_all = []
    for cl in range(0, len(data.neighbors)):
        if subseq_count[cl][1] == max_subseq_count:
            new_neighbors_all.append(data.neighbors[cl].copy())
            continue
        neighbors = data.neighbors[cl].copy()
        need_new_neighbors = (max_subseq_count - subseq_count[cl][1])
        need_double_new = need_new_neighbors - subseq_count[cl][1] if need_new_neighbors - subseq_count[cl][
            1] > 0 else 0
        need_new_neighbors -= need_double_new
        for i in range(0, need_new_neighbors):
            new_neighbor = neighbors[i]
            new_neighbor[random.randint(0, len(neighbors[i]) - 1)] *= 1 + random.uniform(-e, e)
            neighbors.append(new_neighbor)
            if need_double_new > 0:
                new_neighbor = neighbors[i]
                new_neighbor[random.randint(0, len(neighbors[i]) - 1)] *= 1 + random.uniform(-e, e)
                neighbors.append(new_neighbor)
                need_double_new -= 1
        new_neighbors_all.append(neighbors)        
        
    data['neighbors'] = new_neighbors_all
    return data
    #return data

In [91]:
d = augmentation(df)

<ipython-input-90-09b0627e6f21>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  subseq_count = [(i, len(np.array(data.neighbors.iloc[i]))) for i in range(0, len(data.neighbors))]


In [92]:
d

,Unnamed: 0,key,snippet,fraction,neighbors
0,0,0,[0.55714116 0.56319213 0.56522101 0.57919439 0...,0.311884,"[[0.014993, -0.043289, -0.0039098, -0.038406, ..."
1,1,1,[0.47102721 0.44296578 0.4652721 0.50352454 0...,0.430870,"[[7.0488, 4.9723, 4.6134, 3.3802, 2.4563, 2.16..."
2,2,2,[0.54415993 0.545161 0.54891279 0.55129218 0...,0.257536,"[[-0.038406, 0.001217, 0.0039053, -0.0056399, ..."


In [78]:
df2 = df.copy()

In [77]:
df


,Unnamed: 0,key,snippet,fraction,neighbors
0,0,0,[0.55714116 0.56319213 0.56522101 0.57919439 0...,0.311884,"[[0.014993, -0.043289, -0.0039098, -0.038406, ..."
1,1,1,[0.47102721 0.44296578 0.4652721 0.50352454 0...,0.430870,"[[7.0488, 4.9723, 4.6134, 3.3802, 2.4563, 2.16..."
2,2,2,[0.54415993 0.545161 0.54891279 0.55129218 0...,0.257536,"[[-0.038406, 0.001217, 0.0039053, -0.0056399, ..."


In [79]:
df2

,Unnamed: 0,key,snippet,fraction,neighbors
0,0,0,[0.55714116 0.56319213 0.56522101 0.57919439 0...,0.311884,"[[0.014993, -0.043289, -0.0039098, -0.038406, ..."
1,1,1,[0.47102721 0.44296578 0.4652721 0.50352454 0...,0.430870,"[[7.0488, 4.9723, 4.6134, 3.3802, 2.4563, 2.16..."
2,2,2,[0.54415993 0.545161 0.54891279 0.55129218 0...,0.257536,"[[-0.038406, 0.001217, 0.0039053, -0.0056399, ..."


In [80]:
df2['neighbors'] = d

In [81]:
df2

,Unnamed: 0,key,snippet,fraction,neighbors
0,0,0,[0.55714116 0.56319213 0.56522101 0.57919439 0...,0.311884,"[[0.014993, -0.043289, -0.0039098, -0.038406, ..."
1,1,1,[0.47102721 0.44296578 0.4652721 0.50352454 0...,0.430870,"[[7.0488, 4.9723, 4.6134, 3.3802, 2.4563, 2.16..."
2,2,2,[0.54415993 0.545161 0.54891279 0.55129218 0...,0.257536,"[[-0.038406, 0.001217, 0.0039053, -0.0056399, ..."
